For a more detailed guide refer to `tensorflow` or `pytorch` example or to the documentation on https://clipper.ai.

In [2]:
import logging, xgboost as xgb, numpy as np
from clipper_admin.deployers import python as python_deployer
from clipper_admin import ClipperConnection, DockerContainerManager

In [3]:
clipper_conn = ClipperConnection(DockerContainerManager())
clipper_conn.start_clipper()
clipper_conn.connect()

19-05-30:17:03:47 INFO     [docker_container_manager.py:154] [default-cluster] Starting managed Redis instance in Docker
19-05-30:17:03:49 INFO     [docker_container_manager.py:232] [default-cluster] Metric Configuration Saved at /tmp/tmp_nccqeaw.yml
19-05-30:17:03:50 INFO     [clipper_admin.py:143] [default-cluster] Clipper is running
19-05-30:17:03:50 INFO     [clipper_admin.py:156] [default-cluster] Successfully connected to Clipper cluster at localhost:1337


In [4]:
def get_test_point():
    return [np.random.randint(255) for _ in range(784)]

In [5]:
# Create a training matrix.
dtrain = xgb.DMatrix(get_test_point(), label=[0])
# Create parameters, watchlist, and specify the number of rounds
param = {'max_depth': 2, 'eta': 1, 'silent': 1, 'objective': 'binary:logistic'}
watchlist = [(dtrain, 'train')]
num_round = 2
bst = xgb.train(param, dtrain, num_round, watchlist)

[0]	train-error:0
[1]	train-error:0


In [6]:
def predict(xs):
    return bst.predict(xgb.DMatrix(xs))

In [8]:
# Register an xgboost app.
clipper_conn.register_application('xgboost-app', 'integers', 'default_pred', 100000)

19-05-30:17:04:11 INFO     [clipper_admin.py:220] [default-cluster] Application xgboost-app was successfully registered


In [10]:
# Deploy an xgboost model
python_deployer.deploy_python_closure(clipper_conn, 
                                      name='xgboost-model', 
                                      version=1,
                                      input_type="integers", 
                                      func=predict,
                                      pkgs_to_install=['xgboost'])

19-05-30:17:04:18 INFO     [deployer_utils.py:41] Saving function to /tmp/tmp0bixtp4wclipper
19-05-30:17:04:18 INFO     [deployer_utils.py:51] Serialized and supplied predict function
19-05-30:17:04:18 INFO     [python.py:192] Python closure saved
19-05-30:17:04:18 INFO     [clipper_admin.py:513] [default-cluster] Building model Docker image with model data from /tmp/tmp0bixtp4wclipper
19-05-30:17:04:18 INFO     [clipper_admin.py:518] [default-cluster] Step 1/2 : FROM dockerfile-xgboost
19-05-30:17:04:18 INFO     [clipper_admin.py:518] [default-cluster]  ---> 767f3c26b3ba
19-05-30:17:04:18 INFO     [clipper_admin.py:518] [default-cluster] Step 2/2 : COPY /tmp/tmp0bixtp4wclipper /model/
19-05-30:17:04:18 INFO     [clipper_admin.py:518] [default-cluster]  ---> e92379429ad4
19-05-30:17:04:18 INFO     [clipper_admin.py:518] [default-cluster] Successfully built e92379429ad4
19-05-30:17:04:18 INFO     [clipper_admin.py:518] [default-cluster] Successfully tagged default-cluster-xgboost-model:

In [12]:
clipper_conn.link_model_to_app('xgboost-app', 'xgboost-model')

19-05-30:17:04:35 INFO     [clipper_admin.py:282] [default-cluster] Model xgboost-model is now linked to application xgboost-app


In [14]:
# Get query address
query_address = clipper_conn.get_query_addr()

In [15]:
# Run a query
import requests, json, numpy as np
headers = {"Content-type": "application/json"}
requests.post("http://"+query_address+"/xgboost-app/predict", headers=headers, data=json.dumps({
    "input": get_test_point()})).json()
# The result is the same as in the local prediction!

{'query_id': 0, 'output': 0.3266071, 'default': False}

In [17]:
# Stop everything
clipper_conn.stop_all()

19-05-30:17:06:11 INFO     [clipper_admin.py:1324] [default-cluster] Stopped all Clipper cluster and all model containers
